In [1]:
import requests
from datetime import datetime
from collections import defaultdict

In [2]:
PROMETHEUS_HOST = "localhost:9090"

COUNTER_COUNT = 0
GAUGE_COUNT = 0
HISTOGRAM_COUNT = 0
SUMMARY_COUNT = 0

COUNTER_PASS = 0
GAUGE_PASS = 0
HISTOGRAM_PASS = 0
SUMMARY_PASS = 0

expected_metrics = defaultdict()
entries = []

# read and parse the answer file
expected_answers = open("PrometheusAnswersSecond.csv", "r")
lines = expected_answers.readlines()
expected_answers.close()


def parse_label(label_str):
    label_arr = label_str.replace("{", "").replace("}", "").replace('\n', "").split(":")
    labels = {label_arr[0]: label_arr[1]}
    return labels


def parse_buckets(buckets_str):
    buckets_arr = buckets_str.replace("[", "").replace("]", "") \
        .replace("(", "").replace(")", "").split(",")
    buckets = defaultdict()
    i = 0
    cumulative_count = 0
    while i + 1 < len(buckets_arr):
        cumulative_count += int(buckets_arr[i + 1])
        if buckets_arr[i] == "inf":
            buckets_arr[i] = "+Inf"
        buckets[buckets_arr[i]] = cumulative_count
        i = i + 2
    return buckets


def parse_quantiles(quantiles_str):
    quantile_arr = quantiles_str.replace("[", "").replace("]", "") \
        .replace("(", "").replace(")", "").split(",")
    quantiles = defaultdict()
    i = 0
    while i + 1 < len(quantile_arr):
        quantiles[float(quantile_arr[i])] = int(quantile_arr[i + 1])
        i = i + 2
    return quantiles


def request_and_check_single_value(metric_name, mtype):
    URL = "http://" + PROMETHEUS_HOST + "/api/v1/query?query=" + metric_name
    r = requests.get(URL)
    result = r.json()['data']['result']

    if result:
        actual_metric = result[0]['metric']
        actual_value = result[0]['value'][1]

        if mtype == "counter" or mtype == "gauge":
            actual_name = actual_metric['__name__']
            return float(expected_metrics[actual_name]['value']) == float(actual_value)
        elif mtype == "histogram" or mtype == "summary":
            names = actual_metric['__name__'].split("_")
            actual_name = names[0] + "_" + names[1]
            metric = names[2]
            expected_value = float(expected_metrics[actual_name][metric])
            actual_value = float(actual_value)
            return expected_value == actual_value
    return False


def request_and_check_label(metric_name, mtype):
    # send http request to Prometheus server
    if mtype == "histogram" or mtype == "summary":
        metric_name += "_count"

    URL = "http://" + PROMETHEUS_HOST + "/api/v1/query?query=" + metric_name
    r = requests.get(URL)
    result = r.json()['data']['result']
    if result:
        actual_metric = result[0]['metric']
        actual_name = actual_metric['__name__']
        if mtype == "histogram" or mtype == "summary":
            names = actual_name.split("_")
            actual_name = names[0] + "_" + names[1]
        key = "key" + actual_name.split('_')[0][4:]
        actual_label = result[0]['metric'][key]
        return expected_metrics[actual_name]['labels'][key] == actual_label
    return False


def request_and_check_bucket(metric_name):
    names = metric_name.split("_")
    actual_name = names[0] + "_" + names[1]
    URL = "http://" + PROMETHEUS_HOST + "/api/v1/query?query=" + metric_name
    r = requests.get(URL)
    result = r.json()['data']['result']
    if result:
        all_match = True
        for bucket in result:
            le = bucket['metric']['le']
            if le != "+Inf":
                le = str(int(float(bucket['metric']['le'])))
            if le in expected_metrics[actual_name]['buckets']:
                actual_value = int(bucket['value'][1])
                match = actual_value == expected_metrics[actual_name]['buckets'][le]
                all_match = all_match and match
        return all_match
    return False


def request_and_check_quantile(metric_name):
    actual_name = metric_name
    URL = "http://" + PROMETHEUS_HOST + "/api/v1/query?query=" + metric_name
    r = requests.get(URL)
    result = r.json()['data']['result']
    valid_quantiles = [0, 0.5, 0.9, 0.95, 0.99, 1]
    if result:
        all_match = True
        for quantile in result:
            q = float(quantile['metric']['quantile'])
            if q in valid_quantiles and q in expected_metrics[actual_name]['quantiles']:
                actual_value = float(quantile['value'][1])
                expected_value = expected_metrics[actual_name]['quantiles'][q]
                match = actual_value == expected_value
                all_match = all_match and match
        return all_match
    return False

In [3]:
for line in lines:
    split_result = line.split("|")
    metric_type = split_result[0]
    others = split_result[-1].replace(" ", "")
    other_split = others.split(",")
    name = other_split[0]
    desc = other_split[1]
    labels_str = other_split[-1]

    metric_entry = {name: metric_type}
    entries.append(metric_entry)

    expected_metrics[name] = defaultdict()
    expected_metrics[name]['type'] = metric_type
    if metric_type == "counter":
        COUNTER_COUNT += 1
        val = split_result[1]
        expected_metrics[name]['value'] = val
        expected_metrics[name]['labels'] = parse_label(labels_str)
    elif metric_type == "gauge":
        GAUGE_COUNT += 1
        val = split_result[1]
        expected_metrics[name]['value'] = val
        expected_metrics[name]['labels'] = parse_label(labels_str)
    elif metric_type == "histogram":
        HISTOGRAM_COUNT += 1
        sum = int(split_result[1])
        count = int(split_result[2])
        buckets = parse_buckets(split_result[3])
        expected_metrics[name]['sum'] = sum
        expected_metrics[name]['count'] = count
        expected_metrics[name]['buckets'] = buckets
        expected_metrics[name]['labels'] = parse_label(labels_str)
    elif metric_type == "summary":
        SUMMARY_COUNT += 1
        sum = split_result[1]
        count = split_result[2]
        quantiles = parse_quantiles(split_result[3])
        expected_metrics[name]['sum'] = sum
        expected_metrics[name]['count'] = count
        expected_metrics[name]['quantiles'] = quantiles
        expected_metrics[name]['labels'] = parse_label(labels_str)
    else:
        print("Unknown metric type")

In [4]:
for metric_entry in entries:
    name = list(metric_entry.keys())[0]
    mtype = metric_entry[name]

    # [{'metric': {'__name__': 'name2010_counter', 'instance': 'docker.for.mac.localhost:8080', 'job': 'test_prom_exporter', 'key2010': 'value2010'}, 'value': [1598204478.776, '-2']}]
    if mtype == "counter":
        # send http request to Prometheus server
        if request_and_check_single_value(name, mtype) and request_and_check_label(name, mtype):
            COUNTER_PASS += 1
    elif mtype == "gauge":
        if request_and_check_single_value(name, mtype) and request_and_check_label(name, mtype):
            GAUGE_PASS += 1
    elif mtype == "histogram":
        count_name = name + "_count"
        sum_name = name + "_sum"
        bucket_name = name + "_bucket"
        if request_and_check_single_value(count_name, mtype) and request_and_check_single_value(sum_name, mtype) \
                and request_and_check_label(name, mtype) and request_and_check_bucket(bucket_name):
            HISTOGRAM_PASS += 1
    elif mtype == "summary":
        summary_name = name
        sum_name = name + "_sum"
        count_name = name + "_count"
        if request_and_check_single_value(count_name, mtype) and request_and_check_single_value(sum_name, mtype) \
                and request_and_check_label(name, mtype) and request_and_check_quantile(summary_name):
            SUMMARY_PASS += 1

print("Total number of Counter: " + str(COUNTER_COUNT))
print("Test passed Counter: " + str(COUNTER_PASS))
print("Counter test pass rate: " + str(COUNTER_PASS / COUNTER_COUNT * 100) + "%")

print("Total number of Gauge: " + str(GAUGE_COUNT))
print("Test passed Gauge: " + str(GAUGE_PASS))
print("Gauge test pass rate: " + str(GAUGE_PASS / GAUGE_COUNT * 100) + "%")

print("Total number of Histogram: " + str(HISTOGRAM_COUNT))
print("Test passed Histogram: " + str(HISTOGRAM_PASS))
print("Histogram test pass rate: " + str(HISTOGRAM_PASS / HISTOGRAM_COUNT * 100) + "%")

print("Total number of Summary: " + str(SUMMARY_COUNT))
print("Test passed Summary: " + str(SUMMARY_PASS))
print("Summary test pass rate: " + str(SUMMARY_PASS / SUMMARY_COUNT * 100) + "%")

Total number of Counter: 429
Test passed Counter: 429
Counter test pass rate: 100.0%
Total number of Gauge: 436
Test passed Gauge: 436
Gauge test pass rate: 100.0%
Total number of Histogram: 416
Test passed Histogram: 416
Histogram test pass rate: 100.0%
Total number of Summary: 767
Test passed Summary: 767
Summary test pass rate: 100.0%
